In [1]:
#Data manipulation
import pandas as pd
import numpy as np
import string
import regex as re

#NLP NLTK
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
stop_words = list(stopwords.words("english"))

#NLP spaCy model
import spacy
from spacy.tokenizer import Tokenizer

trf = spacy.load('en_core_web_trf')
trf.tokenizer = Tokenizer(trf.vocab)


lg = spacy.load('en_core_web_lg')
lg.tokenizer = Tokenizer(lg.vocab)

from spacy import displacy

# NLP spaCy Matcher
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

matcher = Matcher(lg.vocab)
pmatcher = PhraseMatcher(lg.vocab)

#NLP word embeddings
import gensim
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

# NLP Lemminflect 
import lemminflect
from lemminflect import getLemma

getLemma('screamed', upos = 'VERB')

# NLP Contractions
import contractions

# Motivation for the following work
## Identifying ambiguous cases


In [67]:
#Creating patterns

keys = [] #list of dicts

for pw in cleaned_pw:
    pattern_key = []
    pw = pw.split(' ')
    for w in pw:
        if w.isalpha() == True:
            pattern_key.append(w)
        else:
            wlist = w.split('-')
            for l in wlist:
                if l.isalpha() == True:
                    pattern_key.append({'LOWER': l})
                else:
                    pattern_key.append({'IS_PUNCT': True, 'OP':'*'})
    keys.append(pattern_key)

In [68]:
path = '/Users/daisyliu/Desktop/Research/UTEA_covid19_metaphors/notebooks/sample_training.txt'

with open(path) as f:
    doc = lg(f.read()) #passing entire text string of file into document

list_docs = []

for i in range(0, len(cleaned_pw)):
    pw = cleaned_pw[i]
    doc1 = lg(pw)
    list_docs.append(doc1)

pmatcher.add('ProblemWords', list_docs)
found_matches = pmatcher(doc)

In [69]:
for match_id, start, end in found_matches:
    string_id = lg.vocab.strings[match_id]
    span = doc[start-10:end+10]
    print(match_id, string_id, start, end, span.text)

3051766393194772593 ProblemWords 2014 2016 
He along with his team "Prashamsa" won "Bale Telipale" stand up comedy contest show in Namma TV for three consecutive years
3051766393194772593 ProblemWords 7386 7388 amusement park's ghost train.
The first printing press was set up as early as in 1674 in Mumbai (Bombay).
The
3051766393194772593 ProblemWords 7739 7741 a faculty from the local diocese.
The musical is set in the magical town of Greenwillow.
The neo-classical-style building was


In [70]:
with open(path) as f:
    sample = f.read()
    sample = sample.replace("\'", '').replace('\n', ' ')

In [71]:
replace_word = []
replace_key = []

for match_id, start, end in found_matches:
    string_id = lg.vocab.strings[match_id]
    span = doc[start:end]
    replace_word.append(span.text)
    replace_key.append(problem_words[cleaned_pw.index(span.text)])
    
#Differentiating between nightmarish use cases: movie SET in X, anxiety SET IN

for i in range(0, len(replace_word)):
    sample1 = sample.replace(replace_word[i], replace_key[i])
    print(replace_word[i], replace_key[i])

stand up stand_up
set up set_up
set in set_in


In [72]:
# full sentence: The musical is set in the magical town of Greenwillow.

sample1[44814:44830] # now trying to disambiguate between "set in" and "set" in!

'set_in the magic'

In [4]:
# Getting problem words

wn_words = []
problem_words = []

for word in wn.words():
    ss = wn.synsets(word, pos = 'v')
    for s in ss:
        sep = '.'
        n = s.name().split(sep, 1)[0]
        if n in wn_words:
            pass
        else:
            wn_words.append(n)

for w in range(0, len(wn_words)):
    if wn_words[w].isalpha() == False:
        problem_words.append(wn_words[w])

print(len(problem_words))
print(problem_words)           

1363
['attach_to', 'play_along', 'carry_through', 'roll_up', 'breeze_through', 'air_out', 'air-condition', 'air-cool', 'put_on', 'lend_oneself', 'border_on', 'set_about', 'wake_up', 'give_voice', 'draw_a_bead_on', 'fall_upon', 'bet_on', 'sleep_together', 'free-base', 'give_birth', 'have_a_bun_in_the_oven', 'go_to_bed', 'get_down', 'belt_out', 'tie_down', 'blaze_away', 'shove_off', 'blow_out', 'film_over', 'trouble_oneself', 'break_in', 'break_dance', 'build_up', 'strong-arm', 'burn_off', 'trip_up', 'focus_on', 'cheat_on', 'care_for', 'go_around', 'clear_up', 'clear-cut', 'close_up', 'issue_forth', 'feel_for', 'boil_down', 'band_together', 'get_in_touch', 'plug_in', 'put_up', 'copper-bottom', 'break_through', 'crack_up', 'barge_in', 'break_down', 'bring_around', 'curl_up', 'write_out', 'switch_off', 'cut_off', 'cut_out', 'scratch_out', 'cut_down', 'make_bold', 'go_steady', 'de-iodinate', 'french-fry', 'get_the_better_of', 'change_shape', 'take_down', 'press_down', 'lay_waste_to', 'break

In [16]:
cleaned_pw = []

for word in problem_words:
    cleaned = word.replace('_', ' ')
    cleaned_pw.append(cleaned)
    
print(cleaned_pw)

['attach to', 'play along', 'carry through', 'roll up', 'breeze through', 'air out', 'air-condition', 'air-cool', 'put on', 'lend oneself', 'border on', 'set about', 'wake up', 'give voice', 'draw a bead on', 'fall upon', 'bet on', 'sleep together', 'free-base', 'give birth', 'have a bun in the oven', 'go to bed', 'get down', 'belt out', 'tie down', 'blaze away', 'shove off', 'blow out', 'film over', 'trouble oneself', 'break in', 'break dance', 'build up', 'strong-arm', 'burn off', 'trip up', 'focus on', 'cheat on', 'care for', 'go around', 'clear up', 'clear-cut', 'close up', 'issue forth', 'feel for', 'boil down', 'band together', 'get in touch', 'plug in', 'put up', 'copper-bottom', 'break through', 'crack up', 'barge in', 'break down', 'bring around', 'curl up', 'write out', 'switch off', 'cut off', 'cut out', 'scratch out', 'cut down', 'make bold', 'go steady', 'de-iodinate', 'french-fry', 'get the better of', 'change shape', 'take down', 'press down', 'lay waste to', 'break up',

In [6]:
# Extracting terms with "oneself", "one's"

_one = [] # hyphenated, uninflected version

for word in problem_words:
    if '_one' in word:
        _one.append(word) #len: 31
        
# Creating cleaned_version of the _one list

_one_clean = [] # uninflected version
_one_verb = [] # VERB in the uninflected version

for word in _one:
    _one_clean.append(word.replace('_', ' '))

for word in _one_clean:
    doc = lg(word)
    for token in doc:
        if token.pos_ == 'VERB':
            _one_verb.append(token.text)
            
print(_one[0:5])
print(_one_clean[0:5])
print(_one_verb[0:5])

['lend_oneself', 'trouble_oneself', "talk_through_one's_hat", 'find_oneself', 'help_oneself']
['lend oneself', 'trouble oneself', "talk through one's hat", 'find oneself', 'help oneself']
['lend', 'talk', 'find', 'help', 'assert']


In [7]:
# Creating spaCy matcher in following function

cap_one_clean = []
_one_period = []
_one_comma = []

for word in _one_clean:
    cap_one_clean.append(word.capitalize())

for word in _one_clean:
    _one_period.append(word + '.')

for word in _one_clean:
    _one_comma.append(word + ',')

_one_docs = []

for word in cap_one_clean + _one_period + _one_comma:
    _one_docs.append(lg(word))

one_matcher = pmatcher.add('PHRASES', _one_docs) # <<< spaCy phrasematcher

In [8]:
# Function to identify _one verbs in use

def get_ones(sentence):
    oneself = ['oneself', 'itself', 'himself', 'herself', 'themselves', 'myself']
    ones = ['his', 'hers', 'their', 'theirs', 'ours', 'your', 'my']
    both = oneself + ones
    s = contractions.fix(sentence)
    s_sep = s.split(' ')
    doc = lg(sentence)
    for token in doc:            
        if token.pos_ == 'VERB':
            inflected = token.text
            i = token.i
            t = doc[i+1].text
            if t not in both:
                pass
            else:
                uninflected = token._.lemma()
                if t in oneself:
                    existing = ' '.join([token.text, t])
                    in_place = '_'.join([uninflected, 'oneself'])
                    s = s.replace(existing, in_place)
                    #print(s)
                else:
                    if t in ones:
                        existing = ' '.join([token.text, t])
                        #print(existing)
                        in_place = ' '.join([uninflected, "one's"])
                        #print(in_place)
                        s = s.replace(existing, in_place)
                        #print(s)
                #print(s)    
                doc_s = lg(s)
                matches = pmatcher(doc_s) # <<< if problem happens, check here first
                if len(matches) == 0:
                    return "None matched!"
                else:
                    match = matches[0]
                    start = match[1]
                    end = match[2]
                    span = '_'.join([token.text for token in doc_s[start:end]]).replace(',', '').replace('.', '')
                    return sentence, span 

In [10]:
test_sentence = "I ate an apple, washed my hands, and washed my hands of the day."

get_ones(test_sentence) # <<< problem: return both instances of the verb

('I ate an apple, washed my hands, and washed my hands of the day.',
 "wash_one's_hands")

In [14]:
# Extracting hyphenated words and creating RegEx expressions

hyphenated = []
hyphenated_reglist = []
reg_container = r'{first}\s?{second}'

for word in problem_words:
    if '-' in word:
        hyphenated.append(word) # len: 190

for word in hyphenated:
    word = word.replace('-', ' ')
    doc = lg(word)
    first = doc[0].text
    second = doc[1].text
    filled_reg = reg_container.format(first = first, second = second)
    hyphenated_reglist.append(filled_reg)

print(hyphenated_reglist)

['air\\s?condition', 'air\\s?cool', 'free\\s?base', 'strong\\s?arm', 'clear\\s?cut', 'copper\\s?bottom', 'de\\s?iodinate', 'french\\s?fry', 'double\\s?space', 'drip\\s?dry', 'freeze\\s?dry', 'face\\s?lift', 'mass\\s?produce', 'penny\\s?pinch', 'nickel\\s?and', 'weight\\s?lift', 'quick\\s?freeze', 'single\\s?foot', 're\\s?create', 'cross\\s?file', 'short\\s?change', 'short\\s?circuit', 'baby\\s?sit', 'snow\\s?blind', 'spread\\s?eagle', 'toilet\\s?train', 'tongue\\s?tie', 'two\\s?time', 'water\\s?wash', 'de\\s?escalate', 'about\\s?face', 'still\\s?hunt', 'hee\\s?haw', 'butt\\s?weld', 'cha\\s?cha', 'co\\s?star', 'court\\s?martial', 'cross\\s?index', 'cross\\s?link', 'dive\\s?bomb', 'disk\\s?jockey', 'drop\\s?kick', 'dry\\s?dock', 'e\\s?mail', 'fee\\s?tail', 'finger\\s?paint', 'fly\\s?fish', 'flim\\s?flam', 'atom\\s?bomb', 'one\\s?step', 'place\\s?kick', 'rough\\s?hew', 'teeter\\s?totter', 'skin\\s?dive', 'skinny\\s?dip', 'spring\\s?clean', 'triple\\s?space', 'fine\\s?tune', 'two\\s?step',

In [17]:
def get_hyphenated(sentence):
    hyphenated_words = []
    s = sentence
    doc = lg(sentence)
    for token in doc:
        s = s.replace(token.text, getLemma(token.text, upos = 'VERB')[0])
    for i in range(0, len(hyphenated_reglist)):
        s = re.subn(hyphenated_reglist[i], hyphenated[i], s)[0]
    for h in hyphenated:
        if h in s:
            hyphenated_words.append(h)
    return sentence, s, hyphenated_words

In [18]:
get_hyphenated('I ate a sandwich, then cosigned a lease with my sister, then slam-dunked until noon.')

('I ate a sandwich, then cosigned a lease with my sister, then slam-dunked until noon.',
 'I eat a sandwich then co-sign a lease with my sister then slam-dunk until noon',
 ['co-sign', 'slam-dunk'])

# Other lists

### dep_prt
Second word's dependency is a particle

### pos_adp 
Second word's POS is an adposition

### pos_adj
Second word's POS is an adjective

### pos_adv
Second word's POS is an adverb

### pos_n 
Second word's POS is an noun

### remaining 
Can't sort